In [4]:

import osmnx as ox
import folium
import pandas as pd
import numpy as np
import utils
import folium_utils

# from IPython.display import display
# pd.set_option('display.max_rows', None)

# 峠道
graph = ox.graph_from_point(center_point=(35.334560, 136.989900)
                                , network_type='drive'
                                , dist=1000
                                , simplify=True
                                , custom_filter='["highway"~"tertiary|secondary|primary|trunk"]')

# グラフデータをGeoDataFrameに変換  
gdf_nodes = ox.graph_to_gdfs(graph, nodes=True, edges=False)
gdf_edges = ox.graph_to_gdfs(graph, nodes=False, edges=True)

# エッジの長さから評価する。
lower_bound_meter = 200
upper_bound_meter = 2000
gdf_edges['length_evaluation'] = np.where(gdf_edges['length'] <= lower_bound_meter, 0,
                                   np.where(gdf_edges['length'] >= upper_bound_meter, 1,
                                            (gdf_edges['length'] - lower_bound_meter) / (upper_bound_meter - lower_bound_meter)))

# エッジからジオメトリーのポイント数を取得する(ポイントが多い = Rがついている道?, エッジが重なっている所もポイント数が多くなるので注意(後々で別の評価関数を実装する予定))
gdf_edges['point_cnt'] = gdf_edges['geometry'].apply(lambda x: len(x.coords))

# エッジの距離とポイント数から曲線率?を求める。
gdf_edges['curvature'] = gdf_edges['point_cnt'] / gdf_edges['length']

# 3座標間の角度を求める
def calculate_angle_between_vectors(A, B, C):
    vector_AB = np.array(B) - np.array(A)
    vector_BC = np.array(C) - np.array(B)
    
    dot_product = np.dot(vector_AB, vector_BC)
    norm_AB = np.linalg.norm(vector_AB)
    norm_BC = np.linalg.norm(vector_BC)
    
    cosine_theta = dot_product / (norm_AB * norm_BC)
    angle_rad = np.arccos(cosine_theta)
    angle_deg = np.degrees(angle_rad)
    return angle_deg

# 座標間の角度の変化の合計値を求める
gdf_edges['geometory_angle_total'] = gdf_edges['geometry'].apply(
    lambda x: sum([calculate_angle_between_vectors(x.coords[i-1], x.coords[i], x.coords[i+1]) for i in range(1, len(x.coords)-1)])
)

# 抽出対象のエッジにフラグを立てる.条件はcurvatureが0.025より上 かつ length_evaluationが0.5以上
gdf_edges['is_target'] = np.where((gdf_edges['curvature'] > 0.025) & (gdf_edges['length_evaluation'] > 0.5) & (gdf_edges['geometory_angle_total'] > 120), 1, 0)

# 逆方向のベクトルを持つエッジを削除する
gdf_edges = utils.drop_duplicate_edge(gdf_edges)
gdf_edges

osmid lanes  \
u          v           key                                           
426691834  3822987058  0                            36707102     2   
           1072275362  0                          1172337939     2   
           10892640497 0                          1172340640     2   
3822987058 10892645426 0    [378937792, 201922397, 36707102]     2   
           10892640497 0                          1172337938     1   

                                    ref                name  \
u          v           key                                    
426691834  3822987058  0            453            県道明治村小牧線   
           1072275362  0             16              多治見犬山線   
           10892640497 0             16              多治見犬山線   
3822987058 10892645426 0    [1012, 453]  [市道富士本線, 県道明治村小牧線]   
           10892640497 0            453            県道明治村小牧線   

                                          highway maxspeed  oneway  reversed  \
u          v           key                                                     
426691834  3822987058  0                secondary       30   False      True   
           1072275362  0                  primary       50   False      True   
           10892640497 0                  primary       50   False     False   
3822987058 10892645426 0    [secondary, tertiary]       30   False      True   
           10892640497 0           secondary_link       30    True     False   

                              length  \
u          v           key             
426691834  3822987058  0      45.945   
           1072275362  0     226.296   
           10892640497 0      31.696   
3822987058 10892645426 0    1167.234   
           10892640497 0      60.522   

                                                                     geometry  \
u          v           key                                                      
426691834  3822987058  0    LINESTRING (136.99303 35.33416, 136.99302 35.3...   
           1072275362  0    LINESTRING (136.99303 35.33416, 136.99244 35.3...   
           10892640497 0    LINESTRING (136.99303 35.33416, 136.99309 35.3...   
3822987058 10892645426 0    LINESTRING (136.99294 35.33456, 136.99266 35.3...   
           10892640497 0    LINESTRING (136.99294 35.33456, 136.99303 35.3...   

                           bridge  length_evaluation  point_cnt  curvature  \
u          v           key                                                   
426691834  3822987058  0      NaN           0.000000          6   0.130591   
           1072275362  0      NaN           0.014609          8   0.035352   
           10892640497 0      NaN           0.000000          6   0.189298   
3822987058 10892645426 0      yes           0.537352         85   0.072822   
           10892640497 0      NaN           0.000000          4   0.066092   

                            geometory_angle_total  is_target  
u          v           key                                    
426691834  3822987058  0                28.127659          0  
           1072275362  0                31.451465          0  
           10892640497 0                24.713966          0  
3822987058 10892645426 0               459.208431          1  
           10892640497 0                 5.599649          0